### tableau - extract embedded tables and SQL from twb or twbx files


# Extract Embedded Datasources from Tableau workbooks

This script extracts the **embedded** datasource information (SQL and tables) from Tableau workbook files (`*.twb`) and XML files (`*.xml`) in a specified directory, and saves the results to a CSV file. Embedded datasource downloads are not supported by the `tableauserverclient` SDK (https://tableau.github.io/server-client-python/docs/). 

This script was written for extracting and storing **embedded** custom SQLs in tableau into a central database. Using the csv file generated, one can then create `dbt` models to create these custom SQLs as views etc in the database and eventually publish them as tableau datasources for workbooks to access. 
 
This script will:
1. Download the Tableau workbook files (`*.twb`, `*.twbx`) using the tableauserverclient python SDK
2. Rename twbx filenames to filename.twbx.zip  
3. `unzip_file()` will unzip the renamed .zip files and get the twb files from the unzipped temp dir and move it to the `dest_dir`.
4. `extract_datasource()` will extract the SQL from `CDATA` section of the `.twb` file and table info from the `relation connection=`. Feel free to change the regex patterns in the last cell to suit your needs.

## Requirements

- Python 3.x
- pandas library
- tableauserverclient python sdk

## Usage
3. Run the script from VSCode or from cli using `jupyter nbconvert --to script --execute tableau_extract_ds.ipynb`.

## Output

The script outputs a CSV file named `tableau_ds_list.csv` in the destination directory, which contains the following columns:

- `filename`: the name of the Tableau workbook file or XML file.
- `match`: the extracted SQL query or datasource table name.

## License

This script is licensed under the MIT License.



In [ ]:
# set the destination directory to save the downloaded workbooks
dest_dir = '/path/to/downloaded_workbooks/'
# print(dest_dir)


In [ ]:
# download all workbooks from the tableau server site
import tableauserverclient as TSC

tableau_auth = TSC.TableauAuth('tab_user', 'tab_pwd', 'site_name')
server = TSC.Server('http://tab_server', use_server_version=True)

with server.auth.sign_in(tableau_auth):
    workbooks, pagination_item = server.workbooks.get()
    for workbook in workbooks:
        # print(workbook.id, workbooks.name)
        server.workbooks.download(workbook.id, filepath=dest_dir, no_extract=True)
server.auth.sign_out()

In [ ]:
# loop over all files in the dest directory and rename twbx to zip
import os
import shutil

for filename in os.listdir(dest_dir):
    try:
        if filename.endswith('.twbx'):
            # print(filename)
            # append file name with .zip
            shutil.move(os.path.join(dest_dir, filename), os.path.join(dest_dir, filename+'.zip'))
    except Exception as e:
        print(e)
        continue

In [ ]:
import re
import pandas as pd
import os
import shutil
import zipfile

# set the source directory to the downloaded workbooks
source_dir = dest_dir

# specify the regular expression patterns to search for in the .twb files
# pattern extracts the SQL from the CDATA section in the .twb file
# pattern2 extracts the datasource table from the connection tag in the .twb file
pattern = re.compile(r'<!\[CDATA\[(.*?)\]\]>', re.DOTALL)
pattern2 = re.compile(r'relation connection=(.{0,220})', re.DOTALL)

# define a function to unzip the downloaded twbx.zip workbooks
# move the .twb files to the source directory
def unzip_file(source_dir):
    # loop through all files in the source directory
    for filename in os.listdir(source_dir):
        try:
            # check if the file is a zip file
            if filename.endswith('.zip'):
                # construct the full path to the zip file
                zip_file = os.path.join(source_dir, filename)
                # extract the contents of the zip file to a temporary directory
                with zipfile.ZipFile(zip_file, 'r') as zip_ref:
                    tmp_dir = zip_file.replace('.zip', '')
                    zip_ref.extractall(tmp_dir)
                # loop through all files in the temporary directory
                for tmp_filename in os.listdir(tmp_dir):
                    # check if the file is a .twb file
                    if tmp_filename.endswith('.twb'):
                        # construct the full path to the .twb file
                        twb_file = os.path.join(tmp_dir, tmp_filename)
                        # move the .twb file to the source directory
                        shutil.move(twb_file, source_dir)
                # remove the temporary directory
                shutil.rmtree(tmp_dir)
        except Exception as e:
            print(e)
            continue

unzip_file(source_dir)

# define a function to extract the datasource information from the .twb files
# write to a CSV file in the destination directory
def extract_datasource(source_dir)-> None:
    # create an empty dataframe to store the results
    df = pd.DataFrame(columns=['filename','match'])
    # loop through all files in the source directory
    for filename in os.listdir(source_dir):
        # check if the file is an XML or twb and extract datasource information
        if filename.endswith(('.xml', '.twb')):
            try:
                source_file = os.path.join(source_dir, filename)
                with open(source_file, 'r') as f:
                    contents = f.read()
                matches = pattern.findall(contents)
                matches = matches + pattern2.findall(contents)
                # print(matches)
                for match in matches:
                    if "type='table'" in match:
                        # print(f"{filename},{match[:150]}")
                        row = {'filename':filename,'match':match[:150].split(' ')[2]}
                        df = pd.concat([df,pd.DataFrame(row, index=[0])],ignore_index=True)
                    elif match.upper().startswith('SELECT'):
                        # print(match)
                        row = {'filename':filename,'match':match}
                        df = pd.concat([df,pd.DataFrame(row, index=[0])],ignore_index=True)
                        # print('***************')
            except Exception as e:
                print(e)
                continue
        # save the results to a CSV file in the destination directory
        dest_file = os.path.join(dest_dir,'tableau_ds_list.csv')
        df.drop_duplicates(inplace=True)
        df.to_csv(dest_file, index=False)

extract_datasource(source_dir)